In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [4]:
# 2. Create a temporary view of the DataFrame.
df = spark.read.csv("./home_sales_revised.csv", sep=",", header=True, ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)
df.createOrReplaceTempView('home_sales')
df.show(10)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import year, avg, format_number

query = """
SELECT 
    YEAR(date) AS year,
    ROUND(AVG(price), 2) AS average_price
FROM 
    home_sales
WHERE 
    bedrooms = 4
GROUP BY 
    YEAR(date)
ORDER BY 
    year
"""

average_price_per_year_sql = spark.sql(query)
average_price_per_year_sql.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
SELECT 
    date_built,
    ROUND(AVG(price), 2) AS average_price
FROM 
    home_sales
WHERE 
    bedrooms = 3 AND bathrooms = 3
GROUP BY 
    date_built
ORDER BY 
    date_built
"""

average_price_per_build_year_sql = spark.sql(query)
average_price_per_build_year_sql.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [7]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
SELECT 
    date_built,
    ROUND(AVG(price), 2) AS average_price
FROM 
    home_sales
WHERE 
    bedrooms = 3 AND 
    bathrooms = 3 AND 
    floors = 2 AND 
    sqft_living >= 2000
GROUP BY 
    date_built
ORDER BY 
    date_built
"""

average_price_per_year_construction_sql = spark.sql(query)
average_price_per_year_construction_sql.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()

query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS average_price
FROM 
    home_sales
WHERE 
    price >= 350000
GROUP BY 
    view
ORDER BY 
    view
"""

average_price_per_view_sql = spark.sql(query)
average_price_per_view_sql.show()

# Calculate and print the run time
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   0|    403848.51|
|   1|    401044.25|
|  10|    401868.43|
| 100|    1026669.5|
|  11|    399548.12|
|  12|    401501.32|
|  13|    398917.98|
|  14|    398570.03|
|  15|     404673.3|
|  16|    399586.53|
|  17|    398474.49|
|  18|    399332.91|
|  19|    398953.17|
|   2|    397389.25|
|  20|    399522.81|
|  21|    399758.88|
|  22|    402022.68|
|  23|    403411.92|
|  24|    400284.92|
|  25|    401298.69|
+----+-------------+
only showing top 20 rows

--- 0.3430001735687256 seconds ---


In [9]:
# 7. Cache the temporary table home_sales.
spark.catalog.cacheTable('home_sales')

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

filter_house_using_cache = spark.sql("""
    SELECT view, FORMAT_NUMBER(AVG(price), 2) as average_price 
    FROM home_sales 
    WHERE price >= 350000 
    GROUP BY view 
    ORDER BY view
""")
filter_house_using_cache.show()

print("--- %s seconds ---" % (time.time() - start_time))

# Delete table from cache
# spark.catalog.uncacheTable('home_sales')

+----+-------------+
|view|average_price|
+----+-------------+
|   0|   403,848.51|
|   1|   401,044.25|
|  10|   401,868.43|
| 100| 1,026,669.50|
|  11|   399,548.12|
|  12|   401,501.32|
|  13|   398,917.98|
|  14|   398,570.03|
|  15|   404,673.30|
|  16|   399,586.53|
|  17|   398,474.49|
|  18|   399,332.91|
|  19|   398,953.17|
|   2|   397,389.25|
|  20|   399,522.81|
|  21|   399,758.88|
|  22|   402,022.68|
|  23|   403,411.92|
|  24|   400,284.92|
|  25|   401,298.69|
+----+-------------+
only showing top 20 rows

--- 0.9744987487792969 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
import os
output_path = "./save_data"
df.write.partitionBy("date_built").parquet(output_path, mode='overwrite')

In [ ]:
# 11. Read the formatted parquet data.
p_df = spark.read.parquet('p_sales')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_sales')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()
start_time = time.time()

filter_house_using_parquet = spark.sql("""
    SELECT view, FORMAT_NUMBER(AVG(price), 2) as average_price 
    FROM p_sales 
    WHERE price >= 350000 
    GROUP BY view 
    ORDER BY view
""")

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('home_sales')